In [ ]:
import datetime as dt

import numpy as np

from cryosat_utils import *
%matplotlib inline

In [ ]:
subplot_size=5

sit_names = ['cs2', 'cs22', 'inp', 'lor', 'hir']
titles = ['CS2_12km', 'CS2_30km', 'CS2SMOS', 'CNN-LR', 'CNN-HR']

p_vec = (10,20,30)

date_delta0, date_delta1 = -2, 2

In [ ]:
dates = [
    dt.datetime(2021,1,3),
    dt.datetime(2021,1,6),
    dt.datetime(2021,1,9),
    dt.datetime(2021,1,12),
    dt.datetime(2021,1,15),
    dt.datetime(2021,1,18),
    dt.datetime(2021,1,21),
    dt.datetime(2021,1,24),
    dt.datetime(2021,1,27),
]


In [ ]:
ano_dates = []
for date0 in dates:
    print(date0)
    cs2_files, sit_inp_name, sit_lor_name, sit_hir_name = get_filenames(date0, date_delta0, date_delta1)
    odf = read_cs2_orbits(cs2_files)
    sit_inp, sit_lor, sit_hir = get_sit_2d(sit_inp_name, sit_lor_name, sit_hir_name)
    rho_ice_new, rho_snow_new  = fit_snow_ice_rho(odf, sit_inp, plot=False)
    odf_masked = get_masked_odf(odf, rho_ice_new, rho_snow_new)
    c, r, h = get_smoothed_resampled_orbits(odf_masked, smth_time='12s', resample_time='6s', min_periods=5, t0=None, t1=None)
    sit_cs2 = rasterize_cs2(c, r, h, sit_inp, ws=2)

    c2, r2, h2 = get_smoothed_resampled_orbits(odf_masked, smth_time='30s', resample_time='6s', min_periods=5, t0=None, t1=None)
    sit_cs22 = rasterize_cs2(c2, r2, h2, sit_inp, ws=5)

    ano = {}
    for arr, name in zip([sit_cs2, sit_cs22, sit_inp, sit_lor, sit_hir], sit_names):
        print(name)
        ano[name] = compute_anomaly(arr)
        
    ano_dates.append(ano)        

In [ ]:
ano_all = {}
for name in sit_names:
    ano_all[name] = []
    for ano in ano_dates:
        ano_all[name].append(ano[name][1])
    ano_all[name] = np.array(ano_all[name])


In [ ]:
arrays = [ano_all[name] for name in sit_names]
print('Common mask')
plot_qq(arrays, titles)
print('Individual mask')
plot_qq(arrays, titles, common_mask=False)

In [ ]:
def compute_precentiles_3d(arr, p_vec, ws=5, stp=1):
    ppp = {}
    v = view_as_windows(arr, (arr.shape[0], ws, ws), stp)
    for p in p_vec:
        print(p)
        ppp[p] = np.nanpercentile(v, p, axis=(3,4,5))[0]

    return ppp

ppp_mean = {}
for name in sit_names:
    print(name)
    ppp_mean[name] = compute_precentiles_3d(ano_all[name], p_vec=(10,20,30), stp=5)

In [ ]:
for p in p_vec:
    print(f'Monthly SIT anomaly P{p}')
    arrays = [ppp_mean[name][p] for name in sit_names]
    plot_maps(arrays, titles, clim=(-0.2, 0), subplot_size=7)